In [1]:
import numpy as np
from astropy.io import ascii
from astropy.table import Table
from matplotlib import pyplot as plt
%matplotlib inline
import os

In [12]:
homedir = os.getenv("HOME")
t = Table.read(homedir+'/github/vfcut.fits',format='ascii')
t_full = Table.read(homedir+'/Desktop/v1-20210308/vf_north_v1_main.fits')

## This notebook aims to automate the identification of cutouts with 2+ sersic objects, with "proximity" defined as the galaxies such that their D25$\times$2.5 apertures intersect. I suppose the first task is to generate an array of each galaxy's D25$\times$2.5 (in arcseconds).

In [39]:
r25 = t['radius']
d25 = r25*2
d25_times2 = d25*2

## Isolate RA and DEC arrays.
(When switching to parent sample, replace "t" with "t_full." Vice versa for subsample.)

In [40]:
ra = t['RA']
dec = t['DEC']
galname = t['VFID']

In [16]:
#  idea uno
#  take first (ra,dec). subtract from every other (ra,dec). If coordinate is within the coords enclosed
#  by the 2*D25 aperture, then second sersic object is found. Otherwise, nein. 

#  idea dos
#  find distance between (ra1,dec1) and (ra2,dec2), applying a trigonometric approximation (i.e., distances
#  are linear). Convert to arcseconds (or convert 2*D25 to degrees). If distance < 2*D25 of central galaxy, 
#  dingdingding.

# I prefer idea dos.

##  Find distance between (ra1,dec1) and (ra2,dec2), applying a trigonometric approximation (i.e., distances are linear). Convert to arcseconds (or convert 2$\times$D25 to degrees). If distance < 2$\times$D25 of central galaxy, dingdingding. Add to dictionary.

In [92]:
group_flag = np.zeros(len(t))
counter=0
group_counter=0
group_dict = {}

for i in range(0,len(t)):
    
    ra1 = ra[i]
    dec1 = dec[i]
    galname1 = galname[i]
    d25_2temp = d25_times2[i]                                    #d25 * 2
    #assign empty list to group key
    group_dict['group{}'.format(group_counter)] = []
    
    for n in range(0,len(t)):
        
        ra2 = ra[n]
        dec2 = dec[n]
        galname2 = galname[n]
        dist = np.sqrt((ra1-ra2)**2 + (dec1-dec2)**2)
        dist = dist*3600.                                        #deg to arcsec
        
        if (int(dist) == 0) | (dist > d25_2temp):
            continue
        
        
        if (dist < d25_2temp):    
            if group_counter != 0:
                prev_counter = int(group_counter)-1
            else: 
                prev_counter = 0
            
            #tidying the below if statements with variable names
            curr_item = group_dict['group{}'.format(group_counter)]
            prev_item = group_dict['group{}'.format(prev_counter)]
            
            #if t[i] is not in the current group, add the galaxy.
            if (galname1 not in curr_item) & (galname1 not in prev_item):
                group_dict['group{}'.format(group_counter)].append(galname1)
            
            #if t[n] is not in the current group, add the galaxy.
            if (galname2 not in curr_item) & (galname2 not in prev_item):
                group_dict['group{}'.format(group_counter)].append(galname2)
    
    #if the current group consists of an empty list, remove the item (will be reassigned above)
    if str(group_dict['group{}'.format(group_counter)]) == '[]':
        group_dict.pop('group{}'.format(group_counter), None) 
    #if galaxies populate the list, then raise the group counter by one to prepare for the next group
    else:
        group_counter+=1

In [93]:
group_dict

{'group0': ['VFID0481', 'VFID0483'],
 'group1': ['VFID1548', 'VFID1552'],
 'group2': ['VFID1791', 'VFID1794'],
 'group3': ['VFID1863', 'VFID1864'],
 'group4': ['VFID1935', 'VFID1936'],
 'group5': ['VFID1961', 'VFID1962'],
 'group6': ['VFID2068', 'VFID2075', 'VFID2076'],
 'group7': ['VFID2371', 'VFID2372'],
 'group8': ['VFID2667', 'VFID2669'],
 'group9': ['VFID2672', 'VFID2679'],
 'group10': ['VFID2726', 'VFID2729'],
 'group11': ['VFID2762', 'VFID2766'],
 'group12': ['VFID3554', 'VFID3543', 'VFID3549'],
 'group13': ['VFID3708', 'VFID3711'],
 'group14': ['VFID3860', 'VFID3854'],
 'group15': ['VFID3862', 'VFID3872'],
 'group16': ['VFID3953', 'VFID3960'],
 'group17': ['VFID4180', 'VFID4182'],
 'group18': ['VFID4256', 'VFID4258'],
 'group19': ['VFID4302', 'VFID4305'],
 'group20': ['VFID4508', 'VFID4468', 'VFID4542'],
 'group21': ['VFID4525', 'VFID4547'],
 'group22': ['VFID4567', 'VFID4514', 'VFID4584', 'VFID4646', 'VFID4667'],
 'group23': ['VFID4660', 'VFID4682', 'VFID4696'],
 'group24': ['

### Below is Scrap.
-------

In [84]:
#arranging all group items in list, per key, into numerical order...if doing so is desired. I reckon not.
for i in range(0,len(group_dict)):
    g = group_dict['group{}'.format(i)]
    g.sort()

In [85]:
#an alternative approach to removing duplicates (which, fortunately, tend to be consecutive)
for i in range(0,len(group_dict)):
    try:
        gal_list_1 = group_dict['group{}'.format(i)]
        gal_list_2 = group_dict['group{}'.format(i+1)]
        gal = gal_list_1[1]
        if (gal_list_1 == gal_list_2) | (gal in group_dict['group{}'.format(i+1)]):
            group_dict.pop('group{}'.format(i+1), None)
            
    except:
            continue

#and reassign group numbers (because group0, group1, usw. is computationally and aesthetically desirable)...
g_new = []
g_old = list(group_dict.keys())
for i in range(0,len(group_dict)):
    g_new.append('group{}'.format(i))
    group_dict[g_new[i]] = group_dict.pop(g_old[i])